# 🚀 KAN-PINN JAX GPU Optimization - Levitador Magnético## Identificación de Parámetros Físicos con Differential Evolution en GPUEste notebook implementa un sistema completo de optimización para identificar los parámetros físicos del levitador magnético usando:- **JAX**: Computación vectorizada en GPU- **Differential Evolution**: Optimización metaheurística completamente paralelizada- **GPU Acceleration**: Evaluación de toda la población en un solo paso### 📋 Contenido del Notebook:1. **Configuración del Entorno** (Colab GPU, dependencias)2. **Carga de Datos** (formato estándar y KAN-PINN)3. **Transferencia a GPU** (pandas → JAX arrays)4. **Modelo Físico** (vectorizado en JAX)5. **Differential Evolution** (optimización GPU)6. **Visualización de Resultados**7. **Guardado y Descarga**8. **Comparativa GPU vs CPU** (opcional)### ⚡ Instrucciones de Uso:1. **Activar GPU en Colab**: `Runtime > Change runtime type > GPU (T4)`2. **Ejecutar todas las celdas**: `Runtime > Run all`3. El notebook se auto-configura y usa datos demo del repositorio---

## 1️⃣ Configuración del EntornoInstalamos dependencias y verificamos que tenemos acceso a GPU.

In [ ]:
# Verificar GPU disponibleimport subprocessimport sysprint("🔍 Verificando GPU...")try:    gpu_info = subprocess.check_output(['nvidia-smi'], text=True)    print("✅ GPU detectada:")    print(gpu_info)except:    print("⚠️  No se detectó GPU. Este notebook funcionará en CPU (más lento).")    print("   Para activar GPU: Runtime > Change runtime type > GPU")

In [ ]:
# Instalar dependencias necesariasprint("📦 Instalando dependencias...\n")# JAX con soporte CUDA!pip install -q "jax[cuda12]>=0.4.20" jaxlib>=0.4.20# Otras librerías necesarias!pip install -q numpy scipy pandas matplotlib tqdmprint("\n✅ Dependencias instaladas correctamente")

In [ ]:
# Verificar instalación de JAX y dispositivos disponiblesimport jaximport jax.numpy as jnpfrom jax import random, jit, vmapfrom jax.lax import scanimport numpy as npimport pandas as pdimport matplotlib.pyplot as pltfrom pathlib import Pathimport jsonimport timefrom datetime import datetimefrom tqdm.auto import tqdmfrom functools import partial# Verificar dispositivos JAXprint("\n🖥️  Dispositivos JAX disponibles:")devices = jax.devices()for i, device in enumerate(devices):    print(f"  [{i}] {device}")# Forzar uso de GPU si está disponibleif any('gpu' in str(d).lower() for d in devices):    print("\n✅ GPU disponible - Los cálculos se ejecutarán en GPU")else:    print("\n⚠️  GPU no disponible - Los cálculos se ejecutarán en CPU")# Configurar matplotlibplt.style.use('seaborn-v0_8-darkgrid')%matplotlib inlineprint("\n✅ Imports completados correctamente")

## 2️⃣ Clonar Repositorio (Opcional)Si estamos en Colab y queremos usar los datos del repositorio, clonamos el repo.

In [ ]:
# Verificar si ya estamos en el repo o necesitamos clonarloimport osREPO_URL = "https://github.com/JRavenelco/levitador-benchmark.git"REPO_DIR = "levitador-benchmark"if os.path.exists(REPO_DIR):    print(f"✅ Repositorio ya existe en {REPO_DIR}")    %cd {REPO_DIR}elif os.path.exists("levitador_benchmark.py"):    print("✅ Ya estamos en el directorio del repositorio")    REPO_DIR = "."else:    print(f"📥 Clonando repositorio desde {REPO_URL}...")    !git clone {REPO_URL}    %cd {REPO_DIR}    print("✅ Repositorio clonado correctamente")# Mostrar estructura de datos disponiblesprint("\n📁 Archivos de datos disponibles:")if os.path.exists("data"):    !ls -lh data/*.txt 2>/dev/null || echo "  (No hay archivos .txt en data/)"    if os.path.exists("data/sesiones_kan_pinn"):        print("\n📁 Datos KAN-PINN disponibles:")        !ls -lh data/sesiones_kan_pinn/*.txt 2>/dev/null || echo "  (No hay archivos)"else:    print("  ⚠️  Carpeta 'data' no encontrada")

## 3️⃣ Funciones de Carga de DatosFunciones flexibles para cargar datos en diferentes formatos:- **Formato estándar**: `datos_levitador.txt` (columnas: t, y, i, u, ...)- **Formato KAN-PINN**: `sesiones_kan_pinn/dataset_*.txt` (columnas: t, y, y_obs, dy_obs, i, u, yd)

In [ ]:
# Regex pattern for whitespace separation
WHITESPACE_PATTERN = r"\\s+"

def load_standard_data(filepath, subsample=1):    """    Carga datos en formato estándar del levitador.        Formato esperado (columnas separadas por tabs/espacios):    t [s]  |  y [m]  |  i [A]  |  u [V]  |  dy [m/s]  |  yd [m]        Args:        filepath: Ruta al archivo de datos        subsample: Factor de submuestreo (1=todos, 10=cada 10 puntos)        Returns:        dict con arrays numpy: 't', 'y', 'i', 'u', 'dy' (opcional), 'yd' (opcional)    """    print(f"\n📂 Cargando datos desde: {filepath}")        # Detectar si tiene encabezado    with open(filepath, 'r') as f:        first_line = f.readline().strip()        skiprows = 1 if (first_line.startswith('#') or not first_line[0].isdigit()) else 0        # Cargar con pandas para manejo robusto    if skiprows > 0:        df = pd.read_csv(filepath, sep='\\s+', comment='#', header=None)    else:        df = pd.read_csv(filepath, sep='\\s+', header=None)        # Submuestreo    if subsample > 1:        df = df.iloc[::subsample].reset_index(drop=True)        print(f"   Submuestreo: {subsample}x (quedaron {len(df)} puntos)")        # Extraer columnas    data = {        't': df.iloc[:, 0].values.astype(np.float32),        'y': df.iloc[:, 1].values.astype(np.float32),        'i': df.iloc[:, 2].values.astype(np.float32),        'u': df.iloc[:, 3].values.astype(np.float32),    }        # Columnas opcionales    if df.shape[1] > 4:        data['dy'] = df.iloc[:, 4].values.astype(np.float32)    if df.shape[1] > 5:        data['yd'] = df.iloc[:, 5].values.astype(np.float32)        print(f"✅ Datos cargados: {len(data['t'])} muestras")    print(f"   Columnas: {list(data.keys())}")    print(f"   Rango temporal: {data['t'][0]:.3f} - {data['t'][-1]:.3f} s")        return datadef load_kanpinn_data(filepath, subsample=1):    """    Carga datos en formato KAN-PINN.        Formato esperado (con encabezado comentado):    # Columnas: t y y_obs dy_obs i u yd    t [s]  |  y [m]  |  y_obs [m]  |  dy_obs [m/s]  |  i [A]  |  u [V]  |  yd [m]        Args:        filepath: Ruta al archivo de datos KAN-PINN        subsample: Factor de submuestreo        Returns:        dict con arrays numpy: 't', 'y', 'y_obs', 'dy_obs', 'i', 'u', 'yd'    """    print(f"\n📂 Cargando datos KAN-PINN desde: {filepath}")        # Leer con pandas (robusto para diferentes encodings)    df = pd.read_csv(filepath, sep='\\s+', comment='#', header=None,                     encoding='utf-8', encoding_errors='ignore')        # Submuestreo    if subsample > 1:        df = df.iloc[::subsample].reset_index(drop=True)        print(f"   Submuestreo: {subsample}x (quedaron {len(df)} puntos)")        # Extraer columnas según formato KAN-PINN    data = {        't': df.iloc[:, 0].values.astype(np.float32),        'y': df.iloc[:, 1].values.astype(np.float32),        'y_obs': df.iloc[:, 2].values.astype(np.float32),        'dy_obs': df.iloc[:, 3].values.astype(np.float32),        'i': df.iloc[:, 4].values.astype(np.float32),        'u': df.iloc[:, 5].values.astype(np.float32),    }        if df.shape[1] > 6:        data['yd'] = df.iloc[:, 6].values.astype(np.float32)        print(f"✅ Datos KAN-PINN cargados: {len(data['t'])} muestras")    print(f"   Columnas: {list(data.keys())}")    print(f"   Rango temporal: {data['t'][0]:.3f} - {data['t'][-1]:.3f} s")        return datadef auto_load_data(filepath, subsample=1):    """    Carga automática detectando el formato.        Detecta si es formato KAN-PINN (más columnas, encabezado específico)    o formato estándar.    """    # Leer primera línea para detectar formato    with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:        lines = [f.readline() for _ in range(10)]        # Buscar indicadores de formato KAN-PINN    is_kanpinn = any('y_obs' in line or 'dy_obs' in line or 'dataset_' in str(filepath)                      for line in lines)        if is_kanpinn:        return load_kanpinn_data(filepath, subsample)    else:        return load_standard_data(filepath, subsample)print("✅ Funciones de carga de datos definidas")

## 4️⃣ Cargar Datos ExperimentalesCargamos los datos del levitador. Por defecto usa datos demo del repositorio.**Para usar tus propios datos:**1. Sube el archivo a Colab (icono de carpeta a la izquierda)2. Cambia `DATA_FILE` por tu ruta3. Re-ejecuta esta celda

In [ ]:
# ========== CONFIGURACIÓN DE DATOS ==========# Opciones de archivo de datos (descomenta el que quieras usar):DATA_FILE = "data/datos_levitador.txt"  # Datos estándar# DATA_FILE = "data/sesiones_kan_pinn/dataset_escalon_20251217_205858.txt"  # KAN-PINN escalón# DATA_FILE = "data/sesiones_kan_pinn/dataset_senoidal_20251217_205952.txt"  # KAN-PINN senoidal# DATA_FILE = "data/sesiones_kan_pinn/dataset_chirp_20251217_210058.txt"  # KAN-PINN chirp# Factor de submuestreo (para acelerar optimización)# 1 = usar todos los puntos (lento pero preciso)# 10 = usar 1 de cada 10 puntos (rápido, suficiente para optimización)SUBSAMPLE = 10# ============================================# Cargar datosdata = auto_load_data(DATA_FILE, subsample=SUBSAMPLE)# Mostrar primeras muestrasprint("\n📊 Primeras 5 muestras:")df_preview = pd.DataFrame({    't [s]': data['t'][:5],    'y [mm]': data['y'][:5] * 1000,    'i [A]': data['i'][:5],    'u [V]': data['u'][:5]})print(df_preview.to_string(index=False))# Estadísticas básicasprint(f"\n�� Estadísticas:")print(f"   Posición: {data['y'].min()*1000:.2f} - {data['y'].max()*1000:.2f} mm")print(f"   Corriente: {data['i'].min():.3f} - {data['i'].max():.3f} A")print(f"   Voltaje: {data['u'].min():.3f} - {data['u'].max():.3f} V")# Plot rápido de los datosfig, axes = plt.subplots(3, 1, figsize=(12, 8))axes[0].plot(data['t'], data['y']*1000, 'b-', linewidth=1)axes[0].set_ylabel('Posición [mm]', fontsize=11)axes[0].set_title('Datos Experimentales Cargados', fontsize=13, fontweight='bold')axes[0].grid(True, alpha=0.3)axes[1].plot(data['t'], data['i'], 'r-', linewidth=1)axes[1].set_ylabel('Corriente [A]', fontsize=11)axes[1].grid(True, alpha=0.3)axes[2].plot(data['t'], data['u'], 'g-', linewidth=1)axes[2].set_ylabel('Voltaje [V]', fontsize=11)axes[2].set_xlabel('Tiempo [s]', fontsize=11)axes[2].grid(True, alpha=0.3)plt.tight_layout()plt.show()print("\n✅ Datos cargados y visualizados correctamente")

## 5️⃣ Transferir Datos a GPU (JAX)Convertimos los arrays de numpy a JAX arrays para aprovechar la GPU.

In [ ]:
# Transferir datos a JAX (automáticamente a GPU si está disponible)print("🔄 Transfiriendo datos a GPU...")# Convertir a JAX arrayst_jax = jnp.array(data['t'])y_jax = jnp.array(data['y'])i_jax = jnp.array(data['i'])u_jax = jnp.array(data['u'])print(f"✅ Datos en GPU:")print(f"   t_jax: {t_jax.shape} en {t_jax.device()}")print(f"   y_jax: {y_jax.shape} en {y_jax.device()}")print(f"   i_jax: {i_jax.shape} en {i_jax.device()}")print(f"   u_jax: {u_jax.shape} en {u_jax.device()}")# Calcular pasos de tiempodt_jax = jnp.diff(t_jax, prepend=t_jax[0])dt_mean = float(jnp.mean(dt_jax))print(f"\n⏱️  Paso de tiempo promedio: {dt_mean:.4f} s")print(f"   Frecuencia de muestreo: {1/dt_mean:.1f} Hz")

## 6️⃣ Modelo Físico del Levitador (JAX Vectorizado)Definimos el modelo físico del levitador magnético:### Ecuaciones del Sistema:**Inductancia no lineal:**$$L(y) = k_0 + \frac{k}{1 + y/a}$$**Dinámica mecánica:**$$m \ddot{y} = \frac{1}{2} i^2 \frac{dL}{dy} + mg$$**Dinámica eléctrica:**$$L(y) \frac{di}{dt} + \frac{dL}{dy} \dot{y} \cdot i + R \cdot i = u$$### Parámetros a Identificar:- **k0**: Inductancia base [H]- **k**: Coeficiente de inductancia [H]- **a**: Parámetro geométrico [m]

In [ ]:
# Constantes físicas del sistemaM_SPHERE = 0.018  # Masa de la esfera [kg]GRAVITY = 9.81    # Aceleración gravitacional [m/s²]RESISTANCE = 2.72  # Resistencia de la bobina [Ω]print(f"⚙️  Constantes físicas del sistema:")print(f"   Masa esfera: {M_SPHERE*1000:.1f} g")print(f"   Gravedad: {GRAVITY:.2f} m/s²")print(f"   Resistencia: {RESISTANCE:.2f} Ω")@jitdef inductance_jax(y, k0, k, a):    """Calcula inductancia L(y) = k0 + k/(1 + y/a)"""    return k0 + k / (1.0 + y / a)@jitdef dL_dy_jax(y, k0, k, a):    """Calcula dL/dy = -k / (a * (1 + y/a)²)"""    denom = 1.0 + y / a    return -k / (a * denom ** 2)@jitdef magnetic_force_jax(i, y, k0, k, a):    """Calcula fuerza magnética F = 0.5 * i² * dL/dy"""    dL = dL_dy_jax(y, k0, k, a)    return 0.5 * i ** 2 * dL@jitdef simulate_step_jax(state, inputs):    """    Un paso de simulación del levitador usando método de Euler.        Args:        state: [y, dy, i] - posición, velocidad, corriente        inputs: (u, dt, k0, k, a, m, g, R) - voltaje, paso de tiempo, parámetros        Returns:        new_state: [y_new, dy_new, i_new]        outputs: [y_new, i_new] - observables    """    y, dy, i = state    u, dt, k0, k, a, m, g, R = inputs        # Calcular inductancia y su derivada    L = inductance_jax(y, k0, k, a)    dL = dL_dy_jax(y, k0, k, a)        # Fuerza magnética    F_mag = 0.5 * i * i * dL        # Aceleración mecánica: m*ddy = F_mag + m*g    ddy = (F_mag + m * g) / m        # Derivada de corriente: L*di = u - R*i - dL*dy*i    di = (u - R * i - dL * dy * i) / L        # Integración (Euler)    y_new = y + dy * dt    dy_new = dy + ddy * dt    i_new = i + di * dt        # Límites físicos    y_new = jnp.clip(y_new, 0.0, 0.03)  # 0-30mm    i_new = jnp.clip(i_new, 0.0, 5.0)   # 0-5A        new_state = jnp.array([y_new, dy_new, i_new])    outputs = jnp.array([y_new, i_new])        return new_state, outputs@jitdef simulate_trajectory_jax(params, u_data, dt_data, y0, i0, m, g, R):    """    Simula trayectoria completa del levitador.        Args:        params: [k0, k, a] - parámetros a optimizar        u_data: voltajes de entrada [N]        dt_data: pasos de tiempo [N]        y0, i0: condiciones iniciales        m, g, R: constantes físicas        Returns:        y_sim, i_sim: posiciones y corrientes simuladas [N]    """    k0, k, a = params        # Estado inicial    state = jnp.array([y0, 0.0, i0])        # Crear inputs para cada paso    inputs = jnp.stack([        u_data,        dt_data,        jnp.full_like(u_data, k0),        jnp.full_like(u_data, k),        jnp.full_like(u_data, a),        jnp.full_like(u_data, m),        jnp.full_like(u_data, g),        jnp.full_like(u_data, R),    ], axis=1)        # Simular usando scan (eficiente en GPU)    _, outputs = scan(simulate_step_jax, state, inputs)        y_sim = outputs[:, 0]    i_sim = outputs[:, 1]        return y_sim, i_simprint("\n✅ Modelo físico definido (vectorizado en JAX)")print("   Funciones compiladas con JIT para máxima velocidad en GPU")

## 7️⃣ Función de Fitness (Error MSE)Definimos la función objetivo que queremos minimizar:$$\text{MSE} = \frac{1}{N} \sum_{i=1}^{N} \left[ (y_i - \hat{y}_i)^2 + \lambda (i_i - \hat{i}_i)^2 \right]$$Donde:- $y_i$, $i_i$: datos experimentales- $\hat{y}_i$, $\hat{i}_i$: valores simulados- $\lambda$: peso relativo de los errores

In [ ]:
# Peso para balancear error de posición vs corrienteWEIGHT_POSITION = 1.0WEIGHT_CURRENT = 0.1print(f"⚖️  Pesos de la función de fitness:")print(f"   Posición: {WEIGHT_POSITION}")print(f"   Corriente: {WEIGHT_CURRENT}")@jitdef fitness_single_jax(params, u_data, dt_data, y_data, i_data, y0, i0, m, g, R):    """    Calcula fitness (MSE) para un conjunto de parámetros.        Args:        params: [k0, k, a]        u_data, dt_data, y_data, i_data: datos experimentales        y0, i0: condiciones iniciales        m, g, R: constantes físicas        Returns:        fitness: error MSE (menor es mejor)    """    # Simular trayectoria    y_sim, i_sim = simulate_trajectory_jax(params, u_data, dt_data, y0, i0, m, g, R)        # Calcular errores    error_y = jnp.mean((y_data - y_sim) ** 2)    error_i = jnp.mean((i_data - i_sim) ** 2)        # Error ponderado    fitness = WEIGHT_POSITION * error_y + WEIGHT_CURRENT * error_i        return fitness# Vectorizar fitness para evaluar toda la población de una vezfitness_population_jax = jit(vmap(    fitness_single_jax,    in_axes=(0, None, None, None, None, None, None, None, None, None)))print("\n✅ Función de fitness definida")print("   Vectorizada con vmap para evaluar población completa en paralelo")

## 8️⃣ Test de Función de FitnessProbamos la función de fitness con parámetros de ejemplo para verificar que funciona.

In [ ]:
# Condiciones inicialesy0 = float(y_jax[0])i0 = float(i_jax[0])print(f"🧪 Test de función de fitness...")print(f"   Condiciones iniciales: y0={y0*1000:.2f} mm, i0={i0:.3f} A")# Parámetros de prueba (cercanos a valores reales esperados)test_params = jnp.array([    [0.036, 0.0035, 0.005],  # Parámetros típicos    [0.040, 0.0040, 0.006],  # Variación 1    [0.032, 0.0030, 0.004],  # Variación 2])print(f"\n   Evaluando {len(test_params)} conjuntos de parámetros...")# Evaluar fitnessstart_time = time.time()fitness_values = fitness_population_jax(    test_params, u_jax, dt_jax, y_jax, i_jax,    y0, i0, M_SPHERE, GRAVITY, RESISTANCE)elapsed = time.time() - start_timeprint(f"\n✅ Test completado en {elapsed*1000:.2f} ms")print(f"\n   Resultados:")for i, (params, fit) in enumerate(zip(test_params, fitness_values)):    print(f"   [{i+1}] k0={params[0]:.4f}, k={params[1]:.4f}, a={params[2]:.5f} → MSE={fit:.6e}")print(f"\n   Velocidad: {len(test_params)/elapsed:.1f} evaluaciones/segundo")

## 9️⃣ Differential Evolution en GPUImplementamos Differential Evolution completamente vectorizado en JAX.### Algoritmo:1. **Inicialización**: Población aleatoria dentro de los límites2. **Mutación**: Para cada individuo $x_i$, crear mutante $v_i = x_{r1} + F \cdot (x_{r2} - x_{r3})$3. **Cruce**: Mezclar $x_i$ y $v_i$ con probabilidad $CR$4. **Selección**: Mantener el mejor entre $x_i$ y $u_i$5. **Repetir** hasta convergencia### Parámetros:- **pop_size**: Tamaño de población (típicamente 50-200)- **F**: Factor de mutación (0.5-0.9)- **CR**: Probabilidad de cruce (0.7-0.95)- **max_iter**: Número máximo de generaciones

In [ ]:
def differential_evolution_jax(    fitness_fn,    bounds,    pop_size=100,    F=0.8,    CR=0.9,    max_iter=200,    seed=42,    verbose=True):    """    Differential Evolution completamente vectorizado en JAX.        Args:        fitness_fn: Función fitness vectorizada (population -> fitness_array)        bounds: Array [n_params, 2] con límites [min, max]        pop_size: Tamaño de población        F: Factor de mutación        CR: Probabilidad de cruce        max_iter: Número de generaciones        seed: Semilla aleatoria        verbose: Mostrar progreso        Returns:        best_solution: Mejor solución encontrada        best_fitness: Mejor fitness        history: Historia de convergencia    """    key = random.PRNGKey(seed)    n_params = bounds.shape[0]        # Inicializar población    key, subkey = random.split(key)    ranges = bounds[:, 1] - bounds[:, 0]    population = random.uniform(subkey, (pop_size, n_params))    population = population * ranges + bounds[:, 0]        # Evaluar población inicial    fitness = fitness_fn(population)        # Mejor solución    best_idx = jnp.argmin(fitness)    best_solution = population[best_idx]    best_fitness = fitness[best_idx]        # Historia    history = {'best': [float(best_fitness)], 'mean': [float(jnp.mean(fitness))]}        if verbose:        print(f"🚀 Iniciando Differential Evolution")        print(f"   Población: {pop_size}")        print(f"   Parámetros: {n_params}")        print(f"   F={F}, CR={CR}")        print(f"   Fitness inicial: {best_fitness:.6e}\n")        # Función para un paso de DE    def de_step(population, fitness, key):        key1, key2, key3 = random.split(key, 3)                # Mutación: v = a + F * (b - c)        # Seleccionar índices aleatorios diferentes        idx_a = random.randint(key1, (pop_size,), 0, pop_size)        idx_b = random.randint(key2, (pop_size,), 0, pop_size)        idx_c = random.randint(key3, (pop_size,), 0, pop_size)                # Vectores mutantes        mutant = population[idx_a] + F * (population[idx_b] - population[idx_c])                # Cruce binomial        key, subkey = random.split(key)        cross_mask = random.uniform(subkey, (pop_size, n_params)) < CR                # Asegurar al menos un parámetro del mutante        key, subkey = random.split(key)        j_rand = random.randint(subkey, (pop_size,), 0, n_params)        force_mask = jnp.arange(n_params)[None, :] == j_rand[:, None]        cross_mask = cross_mask | force_mask                # Trial vectors        trial = jnp.where(cross_mask, mutant, population)                # Aplicar límites        trial = jnp.clip(trial, bounds[:, 0], bounds[:, 1])                # Evaluar trial        trial_fitness = fitness_fn(trial)                # Selección (greedy)        improved = trial_fitness < fitness        population = jnp.where(improved[:, None], trial, population)        fitness = jnp.where(improved, trial_fitness, fitness)                return population, fitness        # Evolución principal    iterator = tqdm(range(max_iter), desc="Optimizando") if verbose else range(max_iter)        for gen in iterator:        # Step de DE        key, subkey = random.split(key)        population, fitness = de_step(population, fitness, subkey)                # Actualizar mejor        best_idx = jnp.argmin(fitness)        current_best_fitness = fitness[best_idx]                if current_best_fitness < best_fitness:            best_fitness = current_best_fitness            best_solution = population[best_idx]                # Guardar historia        history['best'].append(float(best_fitness))        history['mean'].append(float(jnp.mean(fitness)))                # Actualizar barra de progreso        if verbose and isinstance(iterator, tqdm):            iterator.set_postfix({                'best_fitness': f"{best_fitness:.6e}",                'mean_fitness': f"{jnp.mean(fitness):.6e}"            })        if verbose:        print(f"\n✅ Optimización completada")        print(f"   Mejor fitness: {best_fitness:.6e}")        print(f"   Mejor solución: k0={best_solution[0]:.5f}, k={best_solution[1]:.5f}, a={best_solution[2]:.6f}")        return best_solution, best_fitness, historyprint("✅ Differential Evolution definido")

## 🔟 Ejecutar OptimizaciónEjecutamos Differential Evolution para identificar los parámetros óptimos.**Tiempo estimado:**- GPU T4: ~2-5 minutos (100 individuos, 200 generaciones)- CPU: ~10-30 minutos**Ajustar parámetros** según necesites:- `POP_SIZE`: Mayor = más exploración, más lento- `MAX_ITER`: Más generaciones = mejor convergencia- `BOUNDS`: Límites de búsqueda para [k0, k, a]

In [ ]:
# ========== CONFIGURACIÓN DE OPTIMIZACIÓN ==========# Parámetros de Differential EvolutionPOP_SIZE = 100     # Tamaño de población (50-200)MAX_ITER = 200     # Número de generaciones (100-500)F_MUTATION = 0.8   # Factor de mutación (0.5-0.9)CR_CROSSOVER = 0.9 # Probabilidad de cruce (0.7-0.95)RANDOM_SEED = 42   # Semilla para reproducibilidad# Límites de búsqueda: [k0, k, a]# Basados en física del sistema y valores típicosBOUNDS = jnp.array([    [0.020, 0.100],  # k0: Inductancia base [H]    [0.001, 0.010],  # k: Coeficiente inductancia [H]    [0.003, 0.012],  # a: Parámetro geométrico [m] (3-12mm)])print("⚙️  Configuración de optimización:")print(f"   Población: {POP_SIZE} individuos")print(f"   Generaciones: {MAX_ITER}")print(f"   F={F_MUTATION}, CR={CR_CROSSOVER}")print(f"\n   Límites de búsqueda:")print(f"   k0: [{BOUNDS[0,0]:.3f}, {BOUNDS[0,1]:.3f}] H")print(f"   k:  [{BOUNDS[1,0]:.4f}, {BOUNDS[1,1]:.4f}] H")print(f"   a:  [{BOUNDS[2,0]*1000:.1f}, {BOUNDS[2,1]*1000:.1f}] mm")# ====================================================# Crear función fitness parcial con datos fijosfitness_fn_partial = partial(    fitness_population_jax,    u_data=u_jax,    dt_data=dt_jax,    y_data=y_jax,    i_data=i_jax,    y0=y0,    i0=i0,    m=M_SPHERE,    g=GRAVITY,    R=RESISTANCE)print("\n" + "="*60)print("🚀 INICIANDO OPTIMIZACIÓN EN GPU")print("="*60)# Ejecutar optimizaciónstart_time_opt = time.time()best_params, best_fitness_val, opt_history = differential_evolution_jax(    fitness_fn=fitness_fn_partial,    bounds=BOUNDS,    pop_size=POP_SIZE,    F=F_MUTATION,    CR=CR_CROSSOVER,    max_iter=MAX_ITER,    seed=RANDOM_SEED,    verbose=True)elapsed_opt = time.time() - start_time_optprint("\n" + "="*60)print("✅ OPTIMIZACIÓN COMPLETADA")print("="*60)print(f"\n⏱️  Tiempo total: {elapsed_opt:.2f} s ({elapsed_opt/60:.2f} min)")print(f"\n📊 Resultados Finales:")print(f"   Best fitness (MSE): {best_fitness_val:.6e}")print(f"\n   Parámetros identificados:")print(f"   k0 = {best_params[0]:.6f} H")print(f"   k  = {best_params[1]:.6f} H")print(f"   a  = {best_params[2]:.6f} m ({best_params[2]*1000:.2f} mm)")print(f"\n   Evaluaciones totales: {POP_SIZE * (MAX_ITER + 1)}")print(f"   Evaluaciones/segundo: {POP_SIZE * (MAX_ITER + 1) / elapsed_opt:.1f}")

## 1️⃣1️⃣ Simular con Parámetros ÓptimosSimulamos el sistema usando los parámetros identificados para comparar con datos experimentales.

In [ ]:
# Simular con los mejores parámetrosprint("🔄 Simulando con parámetros óptimos...")y_sim_best, i_sim_best = simulate_trajectory_jax(    best_params, u_jax, dt_jax, y0, i0, M_SPHERE, GRAVITY, RESISTANCE)# Calcular errores finaleserror_y_final = jnp.mean((y_jax - y_sim_best) ** 2)error_i_final = jnp.mean((i_jax - i_sim_best) ** 2)rmse_y = jnp.sqrt(error_y_final)rmse_i = jnp.sqrt(error_i_final)print(f"\n✅ Simulación completada")print(f"\n�� Errores finales:")print(f"   RMSE Posición: {rmse_y*1000:.4f} mm")print(f"   RMSE Corriente: {rmse_i:.4f} A")print(f"   MSE Posición: {error_y_final:.6e}")print(f"   MSE Corriente: {error_i_final:.6e}")

## 1️⃣2️⃣ Visualización de ResultadosGraficamos la evolución de la optimización y la comparación modelo vs datos.

In [ ]:
# Crear figura con múltiples subplotsfig = plt.figure(figsize=(16, 12))# 1. Convergenciaax1 = plt.subplot(3, 2, 1)generations = np.arange(len(opt_history['best']))ax1.semilogy(generations, opt_history['best'], 'b-', linewidth=2, label='Mejor')ax1.semilogy(generations, opt_history['mean'], 'r--', linewidth=1.5, alpha=0.7, label='Media')ax1.set_xlabel('Generación', fontsize=11)ax1.set_ylabel('Fitness (MSE)', fontsize=11)ax1.set_title('Convergencia de Differential Evolution', fontsize=12, fontweight='bold')ax1.legend()ax1.grid(True, alpha=0.3)# 2. Posición: Datos vs Modeloax2 = plt.subplot(3, 2, 2)t_np = np.array(t_jax)y_np = np.array(y_jax)y_sim_np = np.array(y_sim_best)ax2.plot(t_np, y_np * 1000, 'b-', linewidth=1.5, label='Datos Experimentales', alpha=0.7)ax2.plot(t_np, y_sim_np * 1000, 'r--', linewidth=1.5, label='Modelo Identificado')ax2.set_xlabel('Tiempo [s]', fontsize=11)ax2.set_ylabel('Posición [mm]', fontsize=11)ax2.set_title('Posición: Datos vs Modelo', fontsize=12, fontweight='bold')ax2.legend()ax2.grid(True, alpha=0.3)# 3. Corriente: Datos vs Modeloax3 = plt.subplot(3, 2, 3)i_np = np.array(i_jax)i_sim_np = np.array(i_sim_best)ax3.plot(t_np, i_np, 'b-', linewidth=1.5, label='Datos Experimentales', alpha=0.7)ax3.plot(t_np, i_sim_np, 'r--', linewidth=1.5, label='Modelo Identificado')ax3.set_xlabel('Tiempo [s]', fontsize=11)ax3.set_ylabel('Corriente [A]', fontsize=11)ax3.set_title('Corriente: Datos vs Modelo', fontsize=12, fontweight='bold')ax3.legend()ax3.grid(True, alpha=0.3)# 4. Error de Posiciónax4 = plt.subplot(3, 2, 4)error_y_t = (y_np - y_sim_np) * 1000ax4.plot(t_np, error_y_t, 'g-', linewidth=1)ax4.axhline(0, color='k', linestyle='--', linewidth=0.8, alpha=0.5)ax4.fill_between(t_np, error_y_t, alpha=0.3, color='g')ax4.set_xlabel('Tiempo [s]', fontsize=11)ax4.set_ylabel('Error [mm]', fontsize=11)ax4.set_title(f'Error de Posición (RMSE={rmse_y*1000:.4f} mm)', fontsize=12, fontweight='bold')ax4.grid(True, alpha=0.3)# 5. Error de Corrienteax5 = plt.subplot(3, 2, 5)error_i_t = i_np - i_sim_npax5.plot(t_np, error_i_t, 'orange', linewidth=1)ax5.axhline(0, color='k', linestyle='--', linewidth=0.8, alpha=0.5)ax5.fill_between(t_np, error_i_t, alpha=0.3, color='orange')ax5.set_xlabel('Tiempo [s]', fontsize=11)ax5.set_ylabel('Error [A]', fontsize=11)ax5.set_title(f'Error de Corriente (RMSE={rmse_i:.4f} A)', fontsize=12, fontweight='bold')ax5.grid(True, alpha=0.3)# 6. Inductancia L(y)ax6 = plt.subplot(3, 2, 6)y_range = jnp.linspace(0.001, 0.020, 200)L_range = inductance_jax(y_range, best_params[0], best_params[1], best_params[2])ax6.plot(np.array(y_range) * 1000, np.array(L_range) * 1000, 'b-', linewidth=2)ax6.scatter(y_np * 1000, inductance_jax(y_jax, best_params[0], best_params[1], best_params[2]) * 1000,            c='r', s=10, alpha=0.5, label='Puntos experimentales')ax6.set_xlabel('Posición y [mm]', fontsize=11)ax6.set_ylabel('Inductancia L(y) [mH]', fontsize=11)ax6.set_title('Inductancia No Lineal Identificada', fontsize=12, fontweight='bold')ax6.legend()ax6.grid(True, alpha=0.3)plt.tight_layout()plt.show()print("\n✅ Visualizaciones generadas correctamente")

## 1️⃣3️⃣ Guardar y Descargar ResultadosGuardamos los resultados de la optimización en formato JSON y las gráficas.

In [ ]:
# Crear directorio de resultadosimport osresults_dir = "resultados_optimizacion"os.makedirs(results_dir, exist_ok=True)print(f"📁 Guardando resultados en: {results_dir}/")# 1. Guardar parámetros y métricas en JSONresults_dict = {    "timestamp": datetime.now().isoformat(),    "data_file": DATA_FILE,    "subsample": SUBSAMPLE,    "n_samples": len(t_jax),    "optimization": {        "algorithm": "Differential Evolution (JAX GPU)",        "pop_size": POP_SIZE,        "max_iter": MAX_ITER,        "F": F_MUTATION,        "CR": CR_CROSSOVER,        "seed": RANDOM_SEED,        "elapsed_time_s": elapsed_opt,        "evaluations": POP_SIZE * (MAX_ITER + 1),        "evaluations_per_second": POP_SIZE * (MAX_ITER + 1) / elapsed_opt    },    "parameters_identified": {        "k0": float(best_params[0]),        "k": float(best_params[1]),        "a": float(best_params[2])    },    "fitness": {        "best": float(best_fitness_val),        "rmse_position_mm": float(rmse_y * 1000),        "rmse_current_A": float(rmse_i),        "mse_position": float(error_y_final),        "mse_current": float(error_i_final)    },    "convergence_history": {        "best": [float(x) for x in opt_history['best']],        "mean": [float(x) for x in opt_history['mean']]    }}json_path = f"{results_dir}/optimization_results.json"with open(json_path, 'w') as f:    json.dump(results_dict, f, indent=2)print(f"✅ Resultados guardados en: {json_path}")# 2. Guardar gráfica de convergenciafig_conv = plt.figure(figsize=(10, 6))plt.semilogy(opt_history['best'], 'b-', linewidth=2, label='Mejor')plt.semilogy(opt_history['mean'], 'r--', linewidth=1.5, alpha=0.7, label='Media')plt.xlabel('Generación', fontsize=12)plt.ylabel('Fitness (MSE)', fontsize=12)plt.title('Convergencia de la Optimización', fontsize=14, fontweight='bold')plt.legend(fontsize=11)plt.grid(True, alpha=0.3)plt.tight_layout()conv_path = f"{results_dir}/convergencia.png"plt.savefig(conv_path, dpi=150)plt.close()print(f"✅ Gráfica de convergencia guardada en: {conv_path}")# 3. Guardar gráfica de comparaciónfig_comp = plt.figure(figsize=(14, 10))ax1 = plt.subplot(2, 2, 1)ax1.plot(t_np, y_np * 1000, 'b-', linewidth=1.5, label='Experimental', alpha=0.7)ax1.plot(t_np, y_sim_np * 1000, 'r--', linewidth=1.5, label='Modelo')ax1.set_xlabel('Tiempo [s]')ax1.set_ylabel('Posición [mm]')ax1.set_title('Posición: Datos vs Modelo')ax1.legend()ax1.grid(True, alpha=0.3)ax2 = plt.subplot(2, 2, 2)ax2.plot(t_np, i_np, 'b-', linewidth=1.5, label='Experimental', alpha=0.7)ax2.plot(t_np, i_sim_np, 'r--', linewidth=1.5, label='Modelo')ax2.set_xlabel('Tiempo [s]')ax2.set_ylabel('Corriente [A]')ax2.set_title('Corriente: Datos vs Modelo')ax2.legend()ax2.grid(True, alpha=0.3)ax3 = plt.subplot(2, 2, 3)ax3.plot(t_np, error_y_t, 'g-', linewidth=1)ax3.axhline(0, color='k', linestyle='--', alpha=0.5)ax3.fill_between(t_np, error_y_t, alpha=0.3, color='g')ax3.set_xlabel('Tiempo [s]')ax3.set_ylabel('Error [mm]')ax3.set_title(f'Error de Posición (RMSE={rmse_y*1000:.4f} mm)')ax3.grid(True, alpha=0.3)ax4 = plt.subplot(2, 2, 4)L_plot = inductance_jax(y_jax, best_params[0], best_params[1], best_params[2])ax4.scatter(y_np * 1000, np.array(L_plot) * 1000, c='b', s=10, alpha=0.5)y_r = jnp.linspace(0.001, 0.020, 200)L_r = inductance_jax(y_r, best_params[0], best_params[1], best_params[2])ax4.plot(np.array(y_r) * 1000, np.array(L_r) * 1000, 'r-', linewidth=2)ax4.set_xlabel('Posición [mm]')ax4.set_ylabel('Inductancia [mH]')ax4.set_title('Inductancia L(y)')ax4.grid(True, alpha=0.3)plt.tight_layout()comp_path = f"{results_dir}/comparacion_modelo_datos.png"plt.savefig(comp_path, dpi=150)plt.close()print(f"✅ Gráfica de comparación guardada en: {comp_path}")# 4. Descargar archivos (solo funciona en Colab)try:    from google.colab import files    print("\n📥 Descargando resultados...")    files.download(json_path)    files.download(conv_path)    files.download(comp_path)    print("✅ Archivos descargados correctamente")except ImportError:    print("\n⚠️  No estamos en Colab - los archivos están guardados localmente")    print(f"   Puedes encontrarlos en: {results_dir}/")print(f"\n✅ Todos los resultados guardados correctamente")

## 1️⃣4️⃣ Comparativa GPU vs CPU (Opcional)Comparamos la velocidad de evaluación del fitness entre GPU y CPU para demostrar la aceleración.⚠️ **Nota**: Esta celda es opcional y puede tomar unos minutos. Puedes saltarla si solo quieres los resultados.

In [ ]:
print("⏱️  Comparando velocidad GPU vs CPU...")print("   (Esto puede tomar 1-2 minutos)\n")# Crear población de prueban_test = 50test_key = random.PRNGKey(999)test_pop = random.uniform(test_key, (n_test, 3))test_pop = test_pop * (BOUNDS[:, 1] - BOUNDS[:, 0]) + BOUNDS[:, 0]# Benchmark GPUprint("🖥️  Evaluando en GPU...")# Warm-up (compilación JIT)_ = fitness_population_jax(test_pop[:5], u_jax, dt_jax, y_jax, i_jax, y0, i0, M_SPHERE, GRAVITY, RESISTANCE)# Medir tiempo GPUstart_gpu = time.time()for _ in range(5):    _ = fitness_population_jax(test_pop, u_jax, dt_jax, y_jax, i_jax, y0, i0, M_SPHERE, GRAVITY, RESISTANCE)    _.block_until_ready()  # Asegurar que terminaelapsed_gpu = (time.time() - start_gpu) / 5print(f"   Tiempo GPU (promedio 5 runs): {elapsed_gpu*1000:.2f} ms")print(f"   Velocidad: {n_test/elapsed_gpu:.1f} evaluaciones/segundo")# Benchmark CPU (usando numpy)print("\n💻 Evaluando en CPU (simulación secuencial)...")def fitness_cpu_single(params):    """Versión CPU usando numpy"""    k0, k, a = params        # Simular (versión simplificada)    y_sim = []    i_sim = []    y_curr, dy_curr, i_curr = float(y0), 0.0, float(i0)        for idx in range(len(t_jax)):        u = float(u_jax[idx])        dt = float(dt_jax[idx])                # Inductancia        L = k0 + k / (1.0 + y_curr / a)        dL = -k / (a * (1.0 + y_curr / a) ** 2)                # Fuerza        F_mag = 0.5 * i_curr ** 2 * dL                # Dinámica        ddy = (F_mag + M_SPHERE * GRAVITY) / M_SPHERE        di = (u - RESISTANCE * i_curr - dL * dy_curr * i_curr) / L                # Integrar        y_curr = np.clip(y_curr + dy_curr * dt, 0.0, 0.03)        dy_curr = dy_curr + ddy * dt        i_curr = np.clip(i_curr + di * dt, 0.0, 5.0)                y_sim.append(y_curr)        i_sim.append(i_curr)        # Error    y_sim = np.array(y_sim, dtype=np.float32)    i_sim = np.array(i_sim, dtype=np.float32)    y_data_np = np.array(y_jax, dtype=np.float32)    i_data_np = np.array(i_jax, dtype=np.float32)        error_y = np.mean((y_data_np - y_sim) ** 2)    error_i = np.mean((i_data_np - i_sim) ** 2)        return WEIGHT_POSITION * error_y + WEIGHT_CURRENT * error_i# Medir tiempo CPU (solo 10 individuos para no tardar mucho)n_cpu_test = min(10, n_test)start_cpu = time.time()for i in range(n_cpu_test):    _ = fitness_cpu_single(test_pop[i])elapsed_cpu = time.time() - start_cpuprint(f"   Tiempo CPU ({n_cpu_test} evaluaciones): {elapsed_cpu:.2f} s")print(f"   Velocidad: {n_cpu_test/elapsed_cpu:.2f} evaluaciones/segundo")# Comparaciónspeedup = (elapsed_cpu / n_cpu_test) / elapsed_gpu * n_testefficiency_gpu = n_test / elapsed_gpuefficiency_cpu = n_cpu_test / elapsed_cpuprint("\n" + "="*60)print("📊 COMPARATIVA GPU vs CPU")print("="*60)print(f"   Velocidad GPU: {efficiency_gpu:.1f} eval/s")print(f"   Velocidad CPU: {efficiency_cpu:.1f} eval/s")print(f"   🚀 Speedup: {speedup:.1f}x más rápido en GPU")print("="*60)# Gráfico de comparaciónfig, ax = plt.subplots(1, 1, figsize=(10, 6))methods = ['CPU\n(Secuencial)', 'GPU\n(Vectorizado)']speeds = [efficiency_cpu, efficiency_gpu]colors = ['#FF6B6B', '#4ECDC4']bars = ax.bar(methods, speeds, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)# Añadir valores en las barrasfor bar, speed in zip(bars, speeds):    height = bar.get_height()    ax.text(bar.get_x() + bar.get_width()/2., height,            f'{speed:.1f}\neval/s',            ha='center', va='bottom', fontsize=12, fontweight='bold')ax.set_ylabel('Evaluaciones por Segundo', fontsize=13)ax.set_title(f'Comparativa de Velocidad: GPU vs CPU\\n(Speedup: {speedup:.1f}x)',              fontsize=14, fontweight='bold')ax.set_ylim(0, max(speeds) * 1.2)ax.grid(True, axis='y', alpha=0.3)plt.tight_layout()plt.show()print("\n✅ Comparativa completada")

## 🎯 Resumen y Próximos Pasos### ✅ Completado:1. ✅ Configuración del entorno Colab con GPU2. ✅ Carga flexible de datos (formato estándar y KAN-PINN)3. ✅ Transferencia de datos a GPU con JAX4. ✅ Modelo físico vectorizado del levitador5. ✅ Optimización con Differential Evolution en GPU6. ✅ Visualización de resultados y convergencia7. ✅ Guardado de resultados (JSON + gráficas)8. ✅ Comparativa GPU vs CPU### 📊 Resultados Obtenidos:- **Parámetros identificados**: k0, k, a- **Error final (RMSE)**: Posición y Corriente- **Tiempo de optimización**: GPU vs CPU- **Archivos generados**: JSON, gráficas PNG### 🚀 Próximos Pasos:1. **Ajustar parámetros**: Prueba diferentes `POP_SIZE`, `MAX_ITER` para mejorar convergencia2. **Más datos**: Usa otros datasets KAN-PINN (senoidal, chirp, multiescalón)3. **Validación**: Valida los parámetros identificados en nuevos experimentos4. **Modelado avanzado**: Incorpora efectos térmicos, saturación magnética, etc.### 📚 Referencias:- Repositorio: https://github.com/JRavenelco/levitador-benchmark- JAX Documentation: https://jax.readthedocs.io- Differential Evolution: Storn & Price (1997)---**¿Preguntas o mejoras?** Abre un issue en el repositorio de GitHub.¡Gracias por usar este notebook! 🎉